In [ ]:
import random
import time
from copy import copy

import cv2
import numpy as np
from ipycanvas import Canvas, hold_canvas
from IPython.display import clear_output, display
from PIL import Image

In [ ]:
filename='Elk.jpg'

In [ ]:
image = cv2.imread(filename)
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
image = cv2.pyrDown(image)
display(Image.fromarray(image))

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
mid = cv2.Canny(blurred, 30, 150)
display(Image.fromarray(mid))

In [ ]:
world_width = image.shape[1]-1
world_height = image.shape[0]-1
canvas = Canvas(width=world_width, height=world_height)
canvas.fill_style = "#18041f"
canvas.fill_rect(0, 0, world_width, world_height)

In [ ]:
def analyze(c,u,d,l,r,m):
    scores={
       'up': u+random.randint(0,5),
        'down':d+random.randint(0,5),
        'left':l+random.randint(0,5),
        'right':r+random.randint(0,5)
    }
    top =  max(scores, key=lambda e: m-scores.get(e))
    if abs(scores[top] -c) < 5:
        return None
    return top

analyze(10,11,15,13,21,5)

In [ ]:
class Crawler:
    def __init__(self, x, y, base_color=None):
        self.x = x
        self.y = y

        self.ox = copy(x)
        self.oy = copy(y)
        self.life = 500

    def run(self, world):
        global blurred,image
        updates = {}
        
        current = blurred[self.y][self.x] 

        up = mid[self.y-1][self.x] if (self.y-1 < world_height and self.y-1 >0 and self.x < world_width and self.x > 0 ) else 0
        down = mid[self.y+1][self.x] if (self.y+1< world_height and self.y-1 >0 and self.x < world_width and self.x > 0 ) else 0
        left = mid[self.y][self.x+1] if (self.y < world_height and self.y >0 and self.x+1 < world_width and self.x+1 > 0 ) else 0
        right = mid[self.y][self.x-1] if (self.y < world_height and self.y >0 and self.x-1 < world_width and self.x-1 > 0 ) else 0
        meta = [up,down,left,right]
        meta = sum(meta)/len(meta)
        mode = analyze(current,up,down,left,right,meta)

        if mode == 'up':
            self.x += 0
            self.y += -1
            
        elif mode  == 'left':
            self.x += 1
            self.y += 0
            
        elif mode == 'right':
            self.x += -1
            self.y += 0
            
        elif mode == 'down':
            self.x += 0
            self.y += 1
        
        #respawn
        if self.life >0:
            self.life -= 1
        else:
            self.life = 500
            self.x = random.randint(0,world_width)
            self.y = random.randint(0,world_height)

        if self.y < 0:
            self.y = world_height

        if self.y > world_height:
            self.y = 0

        if self.x < 0:
            self.x = world_width

        if self.x > world_width:
            self.x = 0

        if self.x <= 0:
            self.x = world_width
        if current > 10:
            r,g,b = image[self.y][self.x]
            updates[(self.x, self.y)] = f"rgb({r},{g},{b})"
        valid = False
        return updates

In [ ]:
class World:
    data = {}
    actors = []
    data_since_last_update = {}

    def __init__(self, width, height):
        self.width = width
        self.height = height

    def update(self, update_data):
        self.data.update(update_data)
        self.data_since_last_update.update(update_data)

    def updateCanvas(self, canvas):
        for loc in self.data_since_last_update:
            x, y = loc
            canvas.fill_style = self.data_since_last_update[loc]
            canvas.fill_rect(x, y, 1, 1)
        self.data_since_last_update = {}

In [ ]:
ppi = 50
world = World(world_width, world_height)
for y in range(1,ppi):
    for x in range(1,ppi):
        world.actors.append(
            Crawler(
                int((world_width /ppi) *x),
                int((world_height / ppi)*y)
            )
)

In [ ]:
def update(world):
    for ap in world.actors:
        world.update(ap.run(world))
    world.updateCanvas(canvas)

In [ ]:
display(canvas)
update(world)

In [ ]:
import ipywidgets as widgets

count = 0
out = widgets.Output(layout={"border": "1px solid white"})
out

In [ ]:
while count < 6000:
    with hold_canvas(canvas):
        # Clear the old animation step
        # canvas.clear()
        update(world)
        count += 1
        if count % 100 == 0:
            out.clear_output()
            with out:
                print(count)
    # Animation frequency ~50Hz = 1./50. seconds
    time.sleep(0.001)

In [ ]:
canvas.sync_image_data = True

In [ ]:
time.sleep(5)
canvas.to_file(f"{filename}.output.png")
canvas.sync_image_data = False